In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from time import sleep
import pandas as pd
import time
import re
from datetime import date, timedelta


REMOVE_ADS_SCRIPT = """
var ads = document.querySelectorAll('[id^="google_ads"], [class*="ads"], [class*="sponsored"]');
ads.forEach(ad => ad.remove());
"""


def start_google_chrome(url, screen_width=2200, screen_height=1900):
    """
    Undetected Chrome を起動し、指定URLにアクセス
    - ウィンドウを左半分に配置
    - 広告対策オプションを追加
    :param url: 開くURL
    :param screen_width: 画面の横幅 (デフォルト: 2200)
    :param screen_height: 画面の高さ (デフォルト: 1900)
    :return: WebDriver インスタンス
    """

    # Chromeオプションを設定
    options = uc.ChromeOptions()
    options.add_argument("--no-sandbox")  # セキュリティサンドボックス無効化
    options.add_argument(
        "--disable-blink-features=AutomationControlled"
    )  # ボット検出回避
    options.add_argument("--disable-extensions")  # 拡張機能無効化
    options.add_argument("--disable-popup-blocking")  # ポップアップブロック無効化
    options.add_argument("--disable-gpu")  # GPUレンダリング無効化
    options.add_argument("--start-maximized")  # 最大化
    options.add_argument(
        "--disable-infobars"
    )  # "Chromeは自動テストソフトウェアによって制御されています"を非表示
    options.add_argument("--disable-notifications")  # 通知をブロック
    options.add_argument("--remote-debugging-port=9222")  # デバッグ用ポート

    # Chromeドライバーの起動
    driver = uc.Chrome(options=options)

    # 画面の左半分にウィンドウを配置
    driver.set_window_position(0, 0)  # 左上 (0,0) に配置
    driver.set_window_size(screen_width // 2, screen_height)  # 画面の半分の幅に設定

    # 指定URLを開く
    driver.get(url)
    print("-" * 50)
    print(f"実行成功: {url}")
    driver.implicitly_wait(5)  # 最大30秒待機
    time.sleep(1)  # 読み込み待機

    return driver  # WebDriver インスタンスを返す


def click_date_link(driver, DAYS_AGO, MAX_RETRIES=3):
    """
    指定したDAYS_AGOの日付リンクをクリックし、ページ遷移を確認する
    :param driver: Selenium WebDriver
    :param DAYS_AGO: 過去の日数（1=最新の日付）
    :param MAX_RETRIES: 最大リトライ回数
    :return: 成功した場合はTrue、失敗した場合はFalse
    """
    retries = 0
    current_url = driver.current_url

    try:
        wait = WebDriverWait(driver, 10)
        date_links = wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".table-data-cell a"))
        )

        if DAYS_AGO > len(date_links):
            print(f"エラー: DAYS_AGO={DAYS_AGO} に対応するリンクがありません。")
            return False

        target_link = date_links[DAYS_AGO - 1]
        print(f"クリック対象リンク: {target_link.text}")

        while retries < MAX_RETRIES:
            driver.execute_script(
                "arguments[0].scrollIntoView({behavior: 'auto', block: 'center'});",
                target_link,
            )
            time.sleep(2)  # スクロール後の待機
            wait.until(EC.element_to_be_clickable(target_link))
            time.sleep(2)

            target_link.click()

            wait.until(EC.url_changes(current_url))
            time.sleep(2)

            if "google_vignette" in driver.current_url:
                retries += 1
                print(
                    f"⚠ Google広告が表示されました。リトライします ({retries}/{MAX_RETRIES})"
                )
                driver.back()  # 広告ページから戻る
                time.sleep(2)
                continue  # リトライ

            print(f"遷移成功: {driver.current_url}")
            date, hall_name = extract_date_hall(driver.current_url)
            return date, hall_name

    except Exception as e:
        print(f"エラー発生: {e}")

    print("クリック失敗: リトライ回数超過")
    return False  # 失敗


def extract_date_hall(url):
    """
    URL から日付 (YYYY-MM-DD) とホール名を抽出
    :param url: 解析するURL
    :return: (date, hall_name) のタプル
    """
    pattern = r"https://ana-slo\.com/(\d{4})-(\d{1,2})-(\d{1,2})-(.+)-data/"
    match = re.search(pattern, url)

    if match:
        year, month, day, hall_name = match.groups()
        date = f"{year}-{int(month):02d}-{int(day):02d}"  # 月日をゼロ埋め
        hall_name = hall_name.replace(
            "-", "_"
        ).upper()  # "-" をスペースに置き換えて大文字に
        print(f"日付: {date}")
        print(f"ホール名: {hall_name}")
        return date, hall_name
    else:
        return None, None  # マッチしない場合


def click_machine_by_name(driver):
    """
    指定した機種名を検索し、該当する要素をクリック
    :param driver: Selenium WebDriver
    :param search_machine: 検索する機種名
    :return: クリック成功: True / 失敗: False
    """
    try:
        title = driver.title
        date_pattern = r"\d{4}/\d{2}/\d{2}"
        date_match = re.search(date_pattern, title)
        date = date_match.group().replace("/", "-") if date_match else None
        hall_pattern = r"\d{4}/\d{2}/\d{2} (.+?) データまとめ"
        hall_match = re.search(hall_pattern, title)
        hall_name = hall_match.group(1).replace(" ", "_") if hall_match else None

        print(f"日付: {date}")
        print(f"ホール名: {hall_name}")

        # sections = driver.find_elements(By.CLASS_NAME, "unit_get_medals_table a")
        # for i, section in enumerate(sections):
        #     if search_machine in section.text:
        #         machine_name = section.text
        #         section_num = i
        #         # print(f"{i}: {machine_name}")
        #         driver.execute_script(
        #             "arguments[0].scrollIntoView({behavior: 'auto', block: 'end'});",
        #             section,
        #         )
        #         sleep(3)
        #         section.click()
        #         print(f"クリック成功: {machine_name}")
        return date, hall_name

        print("指定の機種が見つかりませんでした。")
        return False

    except Exception as e:
        print(f"❌ エラー発生: {e}")
        return False


def extract_and_save_model_data(driver, hall_name, date):
    """
    指定した機種のデータを抽出し、CSVに保存する
    :param driver: Selenium WebDriver
    :param section_num: セクション番号 (該当機種のインデックス)
    :param hall_name: ホール名
    :param date: 日付 (yyyy-mm-dd)
    :return: 成功時 True, 失敗時 False
    """
    try:
        # すべての機種を表示
        button = driver.find_element(By.ID, "all_data_btn")
        button.click()

        # データ行を取得
        rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, f"#all_data_table_wrapper > div.dataTables_scroll tr")
            ))

        if not rows:
            print(f"⚠️ データが見つかりません")
            # return False

        print(f"{len(rows)} row 取得開始")
        print("取得中...")

        # ヘッダー取得
        header_cells = rows[0].find_elements(By.TAG_NAME, "th")
        columns = [cell.text for cell in header_cells]

        # データ取得
        data = []
        for row in rows[2:]:  # 最終行を除外
            cells = row.find_elements(By.TAG_NAME, "td")
            data.append([cell.text for cell in cells])
        print(f": {len(data)} data 取得完了")

        # DataFrameを作成しCSVに保存
        df = pd.DataFrame(data, columns=columns)
        file_name = f"../csv/{hall_name}_{date}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8-sig")

        print(rows[1].text)

        print(f"データ保存完了: {file_name}")
        return True

    except Exception as e:
        print(f"エラー発生: {e}")
        return False

In [2]:
URL = f"https://ana-slo.com/"
DATA = "ホールデータ"
PREF = "東京都"
HALL_NAME = "exa-first"
PREF = "埼玉県"
HALL_NAME = "パラッツォ川越店"
EXTRA_WORD = "-データ一覧"
url = f"https://ana-slo.com/{DATA}/{PREF}/{HALL_NAME}{EXTRA_WORD}/"

MAX_RETRIES = 5
DAYS_AGO = 1
RERIOD = 1

In [3]:
driver = start_google_chrome("https://google.com")

for days_ago in range(DAYS_AGO, DAYS_AGO+RERIOD):
        
    driver.get(url)
    driver.execute_script(REMOVE_ADS_SCRIPT)
    
    click_date_link(driver, days_ago, MAX_RETRIES=MAX_RETRIES)
    driver.execute_script(REMOVE_ADS_SCRIPT)

    date, hall_name = click_machine_by_name(driver)
    extract_and_save_model_data(driver, hall_name, date)
    
    sleep(5)

driver.close()

--------------------------------------------------
実行成功: https://google.com
クリック対象リンク: 2025/04/03(木)
遷移成功: https://ana-slo.com/2025-04-03-exa-first-data/
日付: 2025-04-03
ホール名: EXA_FIRST
日付: 2025-04-03
ホール名: EXA_FIRST
407 row 取得開始
取得中...
: 405 data 取得完了

データ保存完了: ../csv/EXA_FIRST_2025-04-03.csv


# テストコード

In [3]:
driver = start_google_chrome("https://google.com")
driver.get(url)
driver.execute_script(REMOVE_ADS_SCRIPT)

--------------------------------------------------
実行成功: https://google.com


In [4]:
date, hall_name = click_date_link(driver, DAYS_AGO, MAX_RETRIES=MAX_RETRIES)
driver.execute_script(REMOVE_ADS_SCRIPT)

クリック対象リンク: 2025/04/09(水)
⚠ Google広告が表示されました。リトライします (1/5)
遷移成功: https://ana-slo.com/2025-04-09-%e3%83%91%e3%83%a9%e3%83%83%e3%83%84%e3%82%a9%e5%b7%9d%e8%b6%8a%e5%ba%97-data/
日付: 2025-04-09
ホール名: %E3%83%91%E3%83%A9%E3%83%83%E3%83%84%E3%82%A9%E5%B7%9D%E8%B6%8A%E5%BA%97


In [5]:
# すべての機種表示
button = driver.find_element(By.ID, "all_data_btn")
button.click()

In [6]:
rows = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, f"#all_data_table_wrapper > div.dataTables_scroll tr")
    ))

if not rows:
    print(f"⚠️ データが見つかりません")
    # return False

print(f"{len(rows)} row 取得開始")
print("取得中...")

# # ヘッダー取得
header_cells = rows[0].find_elements(By.TAG_NAME, "th")
columns = [cell.text for cell in header_cells]

# データ取得
data = []
for row in rows[2:]:  # 最終行を除外
    cells = row.find_elements(By.TAG_NAME, "td")
    data.append([cell.text for cell in cells])
print(f": {len(data)} data 取得完了")

322 row 取得開始
取得中...
: 320 data 取得完了


In [121]:
df = pd.DataFrame(data, columns=columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   機種名     320 non-null    object
 1   台番号     320 non-null    object
 2   G数      320 non-null    object
 3   BB      320 non-null    object
 4   RB      320 non-null    object
 5   合成確率    320 non-null    object
 6   BB確率    320 non-null    object
 7   RB確率    320 non-null    object
dtypes: object(8)
memory usage: 20.1+ KB


In [122]:
df.head()

,機種名,台番号,G数,BB,RB,合成確率,BB確率,RB確率
0,アイムジャグラーEX-TP,461,"3,041",13,10,1/132.2,1/233.9,1/304.1
1,アイムジャグラーEX-TP,462,"1,656",5,4,1/184.0,1/331.2,1/414.0
2,アイムジャグラーEX-TP,463,"3,106",12,8,1/155.3,1/258.8,1/388.2
3,アイムジャグラーEX-TP,464,"2,736",13,4,1/160.9,1/210.5,1/684.0
4,アイムジャグラーEX-TP,478,"3,999",16,11,1/148.1,1/249.9,1/363.5


In [123]:
rename_map = {
    "機種名": "machine_name",
    "台番号": "unit_no",
    "G数": "game",
    "差枚": "medals",
    "BB": "BB",
    "RB": "RB"
}

# 存在するカラムだけ抽出 & リネームを一括で行う
available_cols = [col for col in rename_map if col in df.columns]
df_selected = df[available_cols].rename(columns={k: rename_map[k] for k in available_cols})
df_selected['unit_no'] = df_selected['unit_no'].astype(int)
df_selected['game'] = df_selected['game'].str.replace(",", "", regex=False).astype(int)
df_selected['BB'] = df_selected['BB'].astype(int)
df_selected['RB'] = df_selected['RB'].astype(int)

if 'medals' in df_selected.columns:
    df_selected['medals'] = df_selected['medals'].astype(str).str.replace(",", "").astype(int)
    
df_selected.head()

,machine_name,unit_no,game,BB,RB
0,アイムジャグラーEX-TP,461,3041,13,10
1,アイムジャグラーEX-TP,462,1656,5,4
2,アイムジャグラーEX-TP,463,3106,12,8
3,アイムジャグラーEX-TP,464,2736,13,4
4,アイムジャグラーEX-TP,478,3999,16,11


In [31]:
file_name = f"../anaslo_02/csv/{PREF}_{hall_name}_{date}.csv"
df.to_csv(file_name, index=False, encoding="utf-8-sig")

In [29]:

driver.close()